# 1 导入库和设置

In [1]:
import pandas as pd
import re
import os
from IPython.display import display, Markdown

# 设置显示选项
pd.set_option('display.max_colwidth', 100)

# 2 定义机械压缩去词函数

In [5]:
def mechanical_compression(text, max_repeat=3):
    """
    机械压缩去词处理
    :param text: 输入文本
    :param max_repeat: 允许的最大重复次数
    :return: 处理后的文本
    """
    if not isinstance(text, str) or len(text) < 2:
        return text
    
    # 处理连续重复字符
    def process_repeats(s):
        # 匹配2个及以上连续重复的中文字符或英文单词
        pattern = re.compile(r'([\u4e00-\u9fa5])\1{1,}|([a-zA-Z]+\s?)\2{1,}')
        while True:
            match = pattern.search(s)
            if not match:
                break
            matched = match.group()
            # 保留最多max_repeat次重复
            if len(matched) > len(match.group(1) or match.group(2)) * max_repeat:
                replacement = (match.group(1) or match.group(2)) * min(max_repeat, len(matched))
                s = s[:match.start()] + replacement + s[match.end():]
            else:
                break
        return s
    
    # 处理开头和结尾的重复模式
    def process_edges(s):
        # 处理开头重复
        start_pattern = re.compile(r'^([\u4e00-\u9fa5]{1,3})\1+')
        match = start_pattern.match(s)
        if match:
            matched = match.group()
            replacement = match.group(1) * min(max_repeat, len(matched)//len(match.group(1)))
            s = replacement + s[match.end():]
        
        # 处理结尾重复
        end_pattern = re.compile(r'([\u4e00-\u9fa5]{1,3})\1+$')
        match = end_pattern.search(s)
        if match:
            matched = match.group()
            replacement = match.group(1) * min(max_repeat, len(matched)//len(match.group(1)))
            s = s[:match.start()] + replacement
        
        return s
    
    text = process_repeats(text)
    text = process_edges(text)
    return text

# 3 定义处理单个数据集的函数

In [8]:
def process_dataset(input_path, output_path, max_repeat=3):
    """
    对单个数据集进行机械压缩去词处理，并保存结果。
    :param input_path: 输入数据集路径（xlsx 文件）
    :param output_path: 输出数据集路径（xlsx 文件）
    :param max_repeat: 允许的最大重复次数
    """
    # 加载数据集
    print(f"正在处理数据集：{input_path}")
    df = pd.read_excel(input_path)

    # 检查是否有评论列
    if '字段1' not in df.columns:  # 假设评论列名为 '字段1'
        print(f"数据集 {input_path} 中未找到 '字段1' 列，请检查数据格式。")
        return

    # 对评论列应用机械压缩去词
    df['compressed_comment'] = df['字段1'].apply(lambda x: mechanical_compression(x, max_repeat=max_repeat))

    # 保存处理后的数据集
    df.to_excel(output_path, index=False)  # 移除了 encoding 参数
    print(f"处理完成，结果已保存到：{output_path}")

# 4 处理四个数据集

In [10]:
# 定义输出文件夹
output_folder = "compressed_results"

# 如果文件夹不存在，则创建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 定义四个数据集的路径
datasets = [
    {"input": "text_prossessed\阿里健康大药房_cleaned.xlsx", "output": f"{output_folder}/阿里健康大药房_compressed.xlsx"},
    {"input": "text_prossessed\福士德_cleaned.xlsx", "output": f"{output_folder}/福士德_compressed.xlsx"},
    {"input": "text_prossessed\耀典_cleaned.xlsx", "output": f"{output_folder}/耀典_compressed.xlsx"},
    {"input": "text_prossessed\hfine_cleaned.xlsx", "output": f"{output_folder}/hfine_compressed.xlsx"}
]

# 对每个数据集进行处理
for dataset in datasets:
    process_dataset(dataset["input"], dataset["output"])

print(f"所有数据集处理完成！结果保存在文件夹：{output_folder}")

正在处理数据集：text_prossessed\阿里健康大药房_cleaned.xlsx
处理完成，结果已保存到：compressed_results/阿里健康大药房_compressed.xlsx
正在处理数据集：text_prossessed\福士德_cleaned.xlsx
处理完成，结果已保存到：compressed_results/福士德_compressed.xlsx
正在处理数据集：text_prossessed\耀典_cleaned.xlsx
处理完成，结果已保存到：compressed_results/耀典_compressed.xlsx
正在处理数据集：text_prossessed\hfine_cleaned.xlsx
处理完成，结果已保存到：compressed_results/hfine_compressed.xlsx
所有数据集处理完成！结果保存在文件夹：compressed_results
